In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import os
import logging
import argparse
import random
from tqdm import trange, tqdm
import matplotlib as mpl
mpl.use('Agg')

import matplotlib.pyplot as plt

from scipy import interp

from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, auc, confusion_matrix, classification_report
from sklearn.utils.fixes import signature
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torch import nn

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam
#important
from modeling_readmission import BertForSequenceClassification
from modeling_readmission import BertModel

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s', 
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

in the modeling class


In [29]:
notes=pd.read_csv('./data/all_notes_ordered/test.csv')

In [30]:
notes.sample(1)

,Unnamed: 0,HADM_ID,TEXT,OUTPUT_LABEL
268,19023,142204,6:34 pm chest (portable ap) ...,0


In [31]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label
        
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids):#, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
#         self.label_id = label_id
        
def convert_examples_to_features(examples, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

#     label_map = {}
#     for (i, label) in enumerate(label_list):
#         label_map[label] = i

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
#         if example.text_b:
#             tokens_b = tokenizer.tokenize(example.text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        #print (example.label)
#         label_id = label_map[example.label]
#         if ex_index < 5:
#             logger.info("*** Example ***")
#             logger.info("guid: %s" % (example.guid))
#             logger.info("tokens: %s" % " ".join(
#                     [str(x) for x in tokens]))
#             logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
#             logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
#             logger.info(
#                     "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
# #             logger.info("label: %s (id = %d)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids))#,
#                               label_id=label_id))
    return features


In [32]:
args = {
    'bert_model': './model/pretraining',
    'max_seq_length': 256,
    'eval_batch_size': 1
}
model = BertModel.from_pretrained(args['bert_model'])

03/14/2020 06:01:21 - INFO - modeling_readmission -   loading archive file ./model/pretraining
03/14/2020 06:01:21 - INFO - modeling_readmission -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



In [33]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

03/14/2020 06:01:23 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/evendrow/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [34]:
eval_examples = []
for i in range(len(notes)):
    example = []
    note_list = notes.iloc[i].TEXT.split(' "&-o.o-&"  ') # magic separator
    label = notes.iloc[i].OUTPUT_LABEL
    guid = notes.iloc[i].HADM_ID
    for note in note_list:
        example.append(
            InputExample(guid=guid, text_a=note, text_b=None, label=label))
    eval_examples.append(example)

In [35]:
# sanity check
eval_examples[0][1].guid

100133

In [36]:
# convert each example to an array of tokens
# now we have an array of token arrays of different length
eval_features = [convert_examples_to_features(
    example, args['max_seq_length'], tokenizer) for example in eval_examples]

In [37]:
len(eval_features)

593

In [38]:
# model = BertModel.from_pretrained(args.bert_model)
# eval_features = convert_examples_to_features(
#             eval_examples, label_list, args.max_seq_length, tokenizer)
all_input_ids =  [torch.tensor([f.input_ids for f in eval_features[i]], dtype=torch.long) for i in range(len(eval_features))]
all_input_mask = [torch.tensor([f.input_mask for f in eval_features[i]], dtype=torch.long) for i in range(len(eval_features))]
all_segment_ids = [torch.tensor([f.segment_ids for f in eval_features[i]], dtype=torch.long) for i in range(len(eval_features))]
# all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
# eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)#, all_label_ids)
# eval_dataloader = DataLoader(eval_data, sampler=SequentialSampler(eval_data), batch_size=args['eval_batch_size'])



logger.info("***** Running evaluation *****")
logger.info("  Num examples = %d", len(eval_examples))

all_encodings = []
for i in tqdm(range(len(all_input_ids))):#tqdm(eval_features):
    input_ids = all_input_ids[i]
    input_mask = all_input_mask[i]
    segment_ids = all_segment_ids[i]
    encodings = []
    with torch.no_grad():
        _, pooled_output = model(input_ids,segment_ids,input_mask, output_all_encoded_layers=False)
        all_encodings.append(pooled_output)
            

03/14/2020 06:03:14 - INFO - __main__ -   ***** Running evaluation *****
03/14/2020 06:03:14 - INFO - __main__ -     Num examples = 593
100%|██████████| 593/593 [12:29<00:00,  1.26s/it]


In [39]:
len(all_encodings)

593

In [40]:
encodings_numpy = [x.numpy() for x in all_encodings]

In [41]:
encodings_list = [x.tolist() for x in encodings_numpy]

In [42]:
len(encodings_list[1])

5

In [43]:
# Save to file for later use
np.save('processed_notes_test.npy', encodings_list)

In [44]:
# when loading, must do 
# torch.cuda.LongTensor(embedding_list) on each element probably
saved_list = np.load('processed_notes_test.npy', allow_pickle=True)
# pls remember above

In [45]:
# verify
len(saved_list)

593

In [46]:
np.save('processed_notes_labels_test.npy', list(notes.OUTPUT_LABEL))